# test controller

In [1]:
# Test script for controller.jl without using the ROS part
push!(LOAD_PATH,pwd())
using CarMpcUtils
using CarMpc

### Simulation parameters
simLength = 200

# # initialize vehicle
robot = Robot([1.0, 1.0, 0.0, 0.5], model=unicycleDiscrete)

# # tuning parameters
# tuning = Tuning(dt = 0.2, dtMPC = 0.2, N = 20,
#                 Q = [0.5, 0.5, 10.0, 0.0], R = [20.0, 2.0],
#                 P = [1000.0, 20.0], vRef = 10.0, dSafe = 5.0,
#                 eYRef = 0.0, TTC = 3.0, eYSafe = 0.5)
# tuning = Tuning(dt = 1.0, dtMPC = 1.0, N = 2)
# tuning = Tuning(dt = 0.5, N = 10, safe_dis = 1.0, safe_margin = 1.0, cmft_dis = 2.4)
tuning = Tuning()

# map
# Map = TrackMap("maps/RFS_2Lanes_Speed_0128.mat")
field = Field("maps/shared_room_map.mat")

# ### Initialize MPC problem and solve dummy problem
mpc = initializeMPCProblem(robot, field, tuning)
nz, nu, N = mpc.nz, mpc.nu, tuning.N


# ################
# ##### Main #####
# ################

# ### MPC model parameters updated by subscribers
z0 = robot.z
u0 = zeros(nu)
z_h = zeros(4,N+1)

USim = zeros(nu,simLength)
ZSim = [z0 zeros(nz,simLength)]

### Main loop
for t=1:simLength
  ### Update and solve MPC problem
    ZOpt, UOpt, solveTime = updateSolveMpcProblem(mpc, z0, u0, z_h)

  ### Update current input
  u0[1:nu] = UOpt[:,1]

  ### Update ego state
  # updateEgoState!(vehicle,u0,tuning.dtMPC)
  updateEgoState!(rob,u0,tuning.dt)

  ### Variables for logging
  ZSim[:,t+1] = robot.z
  USim[:,t] = u0
end

# using PyPlot
# figure()
# plot(Map.nodes[1,:]',Map.nodes[2,:]')
# plot(ZSim[1,:]',ZSim[2,:]',marker="o")
# show()

maps/shared_room_map.mat

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      144
Number of nonzeros in inequality constraint Jacobian.:      141
Number of nonzeros in Lagrangian Hessian.............:      148

Total number of variables............................:       75
                     variables with only lower bounds:       11
                variables with lower and upper bounds:        0
                     variables with only upper bounds:     

 in depwarn at deprecated.jl:73
 in setValue at deprecated.jl:30
 in map at abstractarray.jl:1340
 in updateSolveMpcProblem at /Users/changliu/Documents/Git/Human_accompanying_robot/exp/ros_julia/scripts/CarMpc.jl:201
 [inlined code] from In[1]:45
 in anonymous at no file:0
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /Users/changliu/.julia/v0.4/IJulia/src/execute_request.jl:183
 in eventloop at /Users/changliu/.julia/v0.4/IJulia/src/IJulia.jl:143
 in anonymous at task.jl:447
while loading In[1], in expression starting on line 43
 in depwarn at deprecated.jl:73
 in setValue at deprecated.jl:30
 in map_to! at abstractarray.jl:1321
 in map at abstractarray.jl:1343
 in updateSolveMpcProblem at /Users/changliu/Documents/Git/Human_accompanying_robot/exp/ros_julia/scripts/CarMpc.jl:201
 [inlined code] from In[1]:45
 in anonymous at no file:0
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /Users/changliu/.julia/v0.4/IJulia/src/execute_request.

LoadError: LoadError: There was a recent **breaking** change in behavior
for solving sequences of nonlinear models.
Previously, users were allowed to modify the data in the model
by modifying the values stored in their own data arrays.
For example:

data = [1.0]
@NLconstraint(m, data[1]*x <= 1)
solve(m)
data[1] = 2.0
solve(m) # coefficient is updated

However, this behavior **no longer works**. Instead,
nonlinear parameters defined with @defNLParam should be used.
See the latest JuMP documentation for more information.
It is possible that this model was exploiting the previous behavior,
and out of extreme caution we have temporarily introduced
this error message.
If you are sure that you are solving the correct model,
then call `EnableNLPResolve()` at the top of this file to disable
this error message.
To return to the last version of JuMP which supported the
old behavior, run `Pkg.pin("JuMP",v"0.11.1")`.

while loading In[1], in expression starting on line 43